# Ejercicios Segunda Parte. Física de Redes Complejas y Aplicaciones Interdisciplinares.
#### Carlos J. Ramos Salas.
En este notebook corresponde al informe a la actividad a entregar en la segunda parte de este curso, en el cual se busca replicar algunos resultados presentados en el artículo: Johnson, S., Domínguez-García, V., Donetti, L., & Muñoz, M. A. (2014). Trophic coherence
determines food-web stability. Proceedings of the National Academy of Sciences, 111(50), 17923-17928. En el cual se estudia el modelo de depredación preferente en una red trófica. Este notebook junto al correspondiente a los ejercicios de la parte anterior puede ser encontrado en el repositorio de [GitHub asociado](https://github.com/ashybabashyba/RedesComplejasUGR) (https://github.com/ashybabashyba/RedesComplejasUGR).